In [26]:
import pandas as pd

In [27]:
df = pd.read_csv('https://raw.githubusercontent.com/edyoda/data-science-complete-tutorial/master/Data/winequality-white.csv', sep=';')

In [28]:
df.quality.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

### Some important observations from above values

1. After the first experiment,we reliaze that we have too less observations to get the right data
2. Can we minimize or bucketize the target information
3. lets Bucketize into 3 classes

In [29]:
def f(r):
    if r <=3:
        return 1
    elif r<=6:
        return 2
    else:
        return 3
    
df['quality'] = df.quality.map(f)      

In [30]:
df.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3633,6.6,0.30,0.25,8.0,0.036,21.0,124.0,0.99362,3.06,0.38,10.8,2
4887,6.2,0.41,0.22,1.9,0.023,5.0,56.0,0.98928,3.04,0.79,13.0,3
2736,6.5,0.17,0.31,1.5,0.041,34.0,121.0,0.99092,3.06,0.46,10.5,2
3866,8.4,0.23,0.32,1.3,0.048,59.0,113.0,0.99178,3.10,0.55,11.0,2
2662,7.0,0.23,0.33,1.0,0.043,46.0,110.0,0.99118,3.04,0.65,10.8,2
1755,6.3,0.25,0.23,14.9,0.039,47.0,142.0,0.99705,3.14,0.35,9.7,2
2232,8.4,0.20,0.31,2.8,0.054,16.0,89.0,0.99416,2.96,0.45,9.5,2
472,6.6,0.24,0.29,2.0,0.023,19.0,86.0,0.99000,3.25,0.45,12.5,2
1773,7.8,0.39,0.26,9.9,0.059,33.0,181.0,0.99550,3.04,0.42,10.9,2
4012,6.4,0.24,0.49,5.8,0.053,25.0,120.0,0.99420,3.01,0.98,10.5,2


Now After the bucketsizing predicting the values using the logistic regression

In [31]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(df.drop(columns = ['quality']), df.quality)

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
le = LogisticRegression()
le.fit(trainX, trainY)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
le.score(testX, testY)

0.8114285714285714

### Now by using the feature selection
if the feature values are continous and target values is descrete in nature or catagorical we have to use Anova univarient method i.e is f_clss

In [37]:
feature_data = df.drop(columns = ['quality'])
target_data = df.quality

In [40]:
from sklearn.feature_selection import f_classif, SelectKBest

In [45]:
ss = SelectKBest(k = 5,score_func = f_classif)

In [47]:
feature_data_f = ss.fit_transform(feature_data, target_data)

In [48]:
trainX, testX, trainY, testY = train_test_split(feature_data_f, target_data)

In [49]:
le.fit(trainX, trainY)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [52]:
le.score(testX, testY)

0.7910204081632654

In [53]:
y_pred = le.predict(testX)

In [54]:
from sklearn.metrics import confusion_matrix

In [59]:
cf = confusion_matrix(y_pred = y_pred, y_true = testY)
cf

array([[  0,   5,   0],
       [  0, 915,  20],
       [  0, 231,  54]], dtype=int64)

In [61]:
pd.DataFrame(cf, columns=['Predicted 1', 'Predicted 2', 'Predicted 3'], index=['Actual 1', 'Actual 2', 'Actual 3'])

,Predicted 1,Predicted 2,Predicted 3
Actual 1,0,5,0
Actual 2,0,915,20
Actual 3,0,231,54


By observing the above confusion matrix column we can write that:
    1. Diagonal count are the correct classification.
2. 199 data actually belongs to class 3 but misclassified as class 2
3. 38 belongs to class 2 but misclassified as class 3
4. 0 data of class 1 is correctly classified
5. 5 data of class 1 is available in test dataset but misclassified as class 2

In [62]:
df.quality.value_counts()

2    3818
3    1060
1      20
Name: quality, dtype: int64

Since from the above value we can see the class 2 have maximum count value hence there is a himgh chance of getting predicted values as class 2
By which the classes seem to be imbalanced
3. As we see from above confusion matrix, because 0 class had very few data model is not at all predicting class 0


### Dealing with imbalanced classes

In [65]:
Most popular technique is generating under representative class.
Either we can repeat information or using nearest neighbours, we generate similar data.
Install a separate library for getting access to new sampling methods - https://imbalanced-learn.readthedocs.io/en/stable/install.html
Other no so popular techniques are reducing over representative class of data

SyntaxError: invalid syntax (<ipython-input-65-3b2b66d1c7fd>, line 1)

In [67]:

from imblearn.over_sampling import RandomOverSampler, SMOTE

ModuleNotFoundError: No module named 'imblearn'